In [4]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import numpy as np
from collections import Counter
import pandas as pd
import seaborn


from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering

from sklearn.cluster import AffinityPropagation
from sklearn import metrics

import time

In [14]:
# Read data
df = pd.read_csv("results.csv")
df.head()

# Remove na columns
print(df.shape)
df=df.dropna(axis=1)
print(df.shape)

# Prep data for clustering
# remove name column
df=df.drop(['name'], axis=1)
print(df.shape)

print(df.head())

(31984, 21)
(31984, 18)
(31984, 17)
     10k  division    25k gender  age  official bib  genderdiv     35k  \
0  17.37         8  47.67      M   47     85.25  W1          8   71.40   
1  32.58         1  82.43      F   33    138.95  F1          1  116.37   
2  16.62         1  45.80      M   41     80.60  W2          1   67.42   
3  32.57         3  82.43      F   24    140.58  F2          3  116.37   
4  17.12         2  46.37      M   40     81.23  W3          2   67.83   

   overall  pace    30k     5k   half    20k country     40k  
0        8  3.27  59.18   8.02  39.72  37.65     JPN   80.43  
1       21  5.30  99.33  16.22  69.47  65.83     KEN  132.10  
2        1  3.08  56.45   7.75  38.03  36.10     RSA   76.10  
3       27  5.37  99.33  16.20  69.47  65.83     ETH  132.95  
4        2  3.10  57.03   8.02  38.60  36.58     JPN   76.72  


In [21]:
df.head()

,10k,division,25k,gender,age,official,bib,genderdiv,35k,overall,pace,30k,5k,half,20k,country,40k
0,17.37,8,47.67,M,47,85.25,W1,8,71.40,8,3.27,59.18,8.02,39.72,37.65,JPN,80.43
1,32.58,1,82.43,F,33,138.95,F1,1,116.37,21,5.30,99.33,16.22,69.47,65.83,KEN,132.10
2,16.62,1,45.80,M,41,80.60,W2,1,67.42,1,3.08,56.45,7.75,38.03,36.10,RSA,76.10
3,32.57,3,82.43,F,24,140.58,F2,3,116.37,27,5.37,99.33,16.20,69.47,65.83,ETH,132.95
4,17.12,2,46.37,M,40,81.23,W3,2,67.83,2,3.10,57.03,8.02,38.60,36.58,JPN,76.72


In [20]:
df.dtypes

10k           object
division       int64
25k           object
gender        object
age            int64
official     float64
bib           object
genderdiv      int64
35k           object
overall        int64
pace         float64
30k           object
5k            object
half          object
20k           object
country       object
40k           object
dtype: object

In [27]:
# convert the remaining categorical data to binaries
df_Cat= pd.get_dummies(df[['country', 'gender','bib']])
cols= df_Cat.columns
df_Cat[cols] = df_Cat[cols].apply(pd.to_numeric, errors='coerce')

In [28]:
df_Cat.head()

,country_ARG,country_AUS,country_AUT,country_BAH,country_BAR,country_BEL,country_BER,country_BLR,country_BOL,country_BRA,...,bib_W44,bib_W46,bib_W48,bib_W49,bib_W50,bib_W51,bib_W6,bib_W7,bib_W8,bib_W9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_Num = df.drop(['country', 'gender','bib'], axis=1)
cols=df_Num.columns
df_Num[cols] = df_Num[cols].apply(pd.to_numeric, errors='coerce')

df_Num.head()

,10k,division,25k,age,official,genderdiv,35k,overall,pace,30k,5k,half,20k,40k
0,17.37,8,47.67,47,85.25,8,71.40,8,3.27,59.18,8.02,39.72,37.65,80.43
1,32.58,1,82.43,33,138.95,1,116.37,21,5.30,99.33,16.22,69.47,65.83,132.10
2,16.62,1,45.80,41,80.60,1,67.42,1,3.08,56.45,7.75,38.03,36.10,76.10
3,32.57,3,82.43,24,140.58,3,116.37,27,5.37,99.33,16.20,69.47,65.83,132.95
4,17.12,2,46.37,40,81.23,2,67.83,2,3.10,57.03,8.02,38.60,36.58,76.72


In [31]:
# combine df_Cat and df_Num
df1=pd.concat([df_Num, df_Cat], axis=1)
df1.shape

(31984, 32078)

# Functions

In [ ]:
def RunMiniBatchKmeans(X_train, X_test, y_train, y_test,n_clusters):
    print()
    print('MiniBatch')
    minibatchkmeans = MiniBatchKMeans(
        init='random',
        n_clusters=n_clusters,
        batch_size=200)
    start_time=time.time()

    minibatchkmeans.fit(X_train)

    # Add the new predicted cluster memberships to the data frame.
    predict_mini = minibatchkmeans.predict(X_test)
    print('Time taken: {} seconds.'.format('%.3f' % (time.time() - start_time)))
    
    # Plot the solution.
    plt.scatter(X_test[:, 0], X_test[:, 1], c=predict_mini)
    plt.show()
    
    df=pd.crosstab(predict_mini, y_test)
    
    print('Comparing Minibatch against the data:\n',pd.crosstab(predict_mini, y_test))
    return(predict_mini)

In [ ]:
def RunSpectralClustering(X_train, X_test, y_train, y_test,n_clusters):
    print()
    print('Spectral clustering')
    start_time=time.time()
    
    # Declare and fit the model.
    sc = SpectralClustering(n_clusters=n_clusters)
    sc.fit(X_train)

    #Predicted clusters.
    predict=sc.fit_predict(X_test)
    
    print('Time taken: {} seconds.'.format('%.3f' % (time.time() - start_time)))
    #Graph results.
    plt.scatter(X_test[:, 0], X_test[:, 1], c=predict)
    plt.show()

    print('Comparing the assigned categories to the ones in the data:')
    print(pd.crosstab(y_test,predict))

In [ ]:
def RunMeanShift(X_train, X_test, y_train, y_test):
    print()
    print('Meanshift')
    start_time=time.time()
    
    bandwidth = estimate_bandwidth(X_train, quantile=0.2, n_samples=500)
    
    print('bandwidth:', bandwidth)
    
    # Declare and fit the model.
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    MS=ms.fit(X_train)
    
    predict_MeanShift=MS.predict(X_test)
    
    # Count number of clusters.
    n_clusters_ = len(np.unique(predict_MeanShift))

    print('Time taken: {} seconds.'.format('%.3f' % (time.time() - start_time)))
    print("Number of estimated clusters: {}".format(n_clusters_))

   
    # Plot the solution.
    plt.scatter(X_test[:, 0], X_test[:, 1], c=predict_MeanShift)
    plt.show()

    print('Comparing the assigned categories to the ones in the data:')
    print(pd.crosstab(y_train,labels))

In [ ]:
def RunAffinityPropagation(X_train, X_test, y_train, y_test):
    print()
    print('Affinity Propagation')
    start_time=time.time()
    
    # Declare the model and fit it in one statement.
    # Note that you can provide arguments to the model, but we didn't.
    af = AffinityPropagation().fit(X_train)
    print('Done')

    labels= af.predict(X_test)
    # Pull the number of clusters and cluster assignments for each data point.
    cluster_centers_indices = af.cluster_centers_indices_
    n_clusters_ = len(np.unique(labels))

    print('Time taken: {} seconds.'.format('%.3f' % (time.time() - start_time)))
    print('Estimated number of clusters: {}'.format(n_clusters_))

    print('Comparing the assigned categories to the ones in the data:')
    print(pd.crosstab(y_test,labels))
    
    plt.figure(1)
    plt.clf()

    # Cycle through each cluster and graph them with a center point for the
    # exemplar and lines from the exemplar to each data point in the cluster.
    colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
    for k, col in zip(range(n_clusters_), colors):
        class_members = labels == k
        cluster_center = X_test[cluster_centers_indices[k]]
        plt.plot(X_test[class_members, 0], X_test[class_members, 1], col + '.')
        plt.plot(cluster_center[0],
                 cluster_center[1],
                 'o',
                 markerfacecolor=col,
                 markeredgecolor='k')
        for x in X_test[class_members]:
            plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col)

    plt.title('Estimated number of clusters: {}'.format(n_clusters_))
    plt.show()

In [ ]:
# 